In [1]:
import pyodbc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pycaret.anomaly import *

In [2]:
## Conexion al DWH
cnxn = pyodbc.connect(
    driver='{SQL Server}',
    server='192.168.100.58',
    uid='bilectura',
    pwd='D1sp@p3l3s')
cursor = cnxn.cursor()
#cnxn.close()

In [3]:
codigo_SQL = "SELECT A.VTAANO AS 'Ano', A.VTAMES AS 'Mes', A.DSTCOD AS 'Codigo distrito', A.VTAZNA AS 'Codigo zona', A.VTANIT AS 'Nit cliente', A.VTAPRDCOD AS 'Codigo producto', B.TPOCOD AS 'Codigo tipo', B.LNACOD AS 'Codigo linea', B.GRPCOD AS 'Codigo grupo', B.CLDCOD AS 'Codigo calidad', SUM(A.VTAVLRVTA) AS 'Ventas', SUM(A.VTACST) AS 'Costo' FROM V_VTA_VTAHEC A LEFT JOIN DIM_PRODUCTOS B ON A.PRMCOD = B.PRMCOD AND A.VTAPRDCOD = B.PRDCOD WHERE A.PRMCOD = 1 AND A.DSTCOD != 30 AND A.VTAZNA < 400 AND A.VTAFCH BETWEEN '2021-01-01' AND '2022-12-31' AND  B.TPOCOD NOT IN (990,999) GROUP BY A.VTAANO, A.VTAMES, A.DSTCOD, A.VTAZNA, A.VTANIT, A.VTAPRDCOD, B.TPOCOD, B.LNACOD, B.GRPCOD, B.CLDCOD"

In [4]:
#Cargue de la data desde el ERP de Dispapeles y se guarda en df
cursor.execute(codigo_SQL)
rows = cursor.fetchall()
df = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])

In [5]:
maestra_gerencias = pd.read_csv('C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Anomaly detection/Maestra_gerencias.csv', delimiter= ";")

In [6]:
df_merged = pd.merge(df, maestra_gerencias, on= "Codigo producto")

In [7]:
df_merged.columns

Index(['Ano', 'Mes', 'Codigo distrito', 'Codigo zona', 'Nit cliente',
       'Codigo producto', 'Codigo tipo', 'Codigo linea', 'Codigo grupo',
       'Codigo calidad', 'Ventas', 'Costo', 'Gerente', 'Línea CAM'],
      dtype='object')

In [9]:
df_gb = df_merged.groupby(["Ano", "Mes", "Codigo distrito", "Codigo zona", "Nit cliente", "Codigo tipo",
                            "Codigo linea", "Codigo grupo", "Codigo calidad", "Gerente", "Línea CAM"]).agg({"Ventas": np.sum,
                                                                                                            "Costo": np.sum})
df_gb = df_gb.reset_index(col_level= 1)
df_gb["Codigo distrito-zona"] = df_gb["Codigo distrito"].astype(str) + "-" + df_gb["Codigo zona"].astype(str)
df_gb

,Ano,Mes,Codigo distrito,Codigo zona,Nit cliente,Codigo tipo,Codigo linea,Codigo grupo,Codigo calidad,Gerente,Línea CAM,Ventas,Costo,Codigo distrito-zona
0,2021,1,10,1,52798044,235,103,180,334,Elementos de Protección Personal,Protección personal,284227.9900,224390,10-1
1,2021,1,10,1,79571067,180,160,59,25,Comunicación visual,Tintas,832743.0000,669435,10-1
2,2021,1,10,1,79743894,180,160,59,25,Comunicación visual,Tintas,278095.0000,222240,10-1
3,2021,1,10,1,800032769,105,1,1,50,Cartulina,Propal,16182936.0000,14611847,10-1
4,2021,1,10,1,800032769,160,168,203,73,Flexografía,Insumos Flexografía,322407.2000,291087,10-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364026,2022,12,90,97,901667148,235,126,179,315,Limpieza y Desinfección,Cuidado Institucional,771.0000,7707,90-97
364027,2022,12,90,97,901667148,235,126,179,319,Limpieza y Desinfección,Cuidado Institucional,1067.0000,10669,90-97
364028,2022,12,90,97,901667148,240,170,13,34,Tecnología Institucional,Tecnología institucional,17976.0000,179795,90-97
364029,2022,12,90,97,901667148,240,170,13,35,Tecnología Institucional,Tecnología institucional,56800.0000,568161,90-97


In [ ]:
list_dst_znas = (df_gb["Codigo distrito"].astype(str) + "-" + df_gb["Codigo zona"].astype(str)).unique()
categorical_columns = ["Ano", "Mes", "Codigo distrito", "Codigo zona", "Nit cliente","Codigo tipo", "Codigo linea",
                        "Codigo grupo", "Codigo calidad", "Gerente", "Línea CAM", "Codigo distrito-zona"]
ignore_columns = ["Ano", "Mes"]

In [ ]:
consolidado = pd.DataFrame([])
for dtozna in list_dst_znas:
    #Creacion del subdataset
    df_dto = df_gb[df_gb["Codigo distrito-zona"] == dtozna]
    
    #Setup modelo
    exp_ano101 = setup(df_dto, 
                        session_id = 42,
                        silent=True,
                        categorical_features = categorical_columns,
                        numeric_features = ["Ventas", "Costo"],
                        ignore_features = ignore_columns,
                        normalize = True,
                        normalize_method = 'robust',
                        verbose = False)
    
    #Modelo iforest
    iforest = create_model('iforest')
    iforest_results = assign_model(iforest)
    resultado_iforest = iforest_results.loc[(iforest_results["Anomaly"] == 1) & 
                                            (iforest_results["Anomaly_Score"] >= 0.04) &
                                            (iforest_results["Ventas"] >= 1000000) &
                                            (iforest_results["Ano"] == 2022) & 
                                            (iforest_results["Mes"] >= 11)
                                            ]

    #Append a una base general
    consolidado = consolidado.append(resultado_iforest)

#Descarga de la base general
consolidado.index = pd.RangeIndex(len(consolidado.index))
consolidado.to_csv(f"C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Anomaly detection/Anomalies_iforest/Anomalies nov + costo 24-1-23.csv",
                    encoding= 'utf-8', index= False, sep= ";", decimal = ",")